In [1]:
import pandas as pd
import numpy as np
import requests
import json
import json
import math
import csv
# import seaborn as sns
import time    
#access data for Riot API and Discord Webhook


In [2]:
#Quick way to execute GET requests
api_key = "RGAPI-a0e0d5f3-801c-43d3-a182-02ae0bc815dc"
def get_req(url, params):
    headers = {
                "X-Riot-Token": api_key,
                }
    response = requests.request("GET", url, headers=headers, params=params)
    return(response.text)

In [3]:
#get a recent match list 
def get_matches(count, puuid, startTime): 
    parameters = {
        "count": count,
        "type": "ranked",
        "startTime": startTime
    }
    url = "https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/" + str(puuid) + "/ids"
    res = get_req(url=url, params=parameters)
    match_list =(res)
    return json.loads(match_list)
    


In [4]:
def get_match_analytics(match_id):
    url = f'https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}'
    res = json.loads(get_req(url, params = None))
    return res

In [32]:
#lets carve out a really big data frame :)
df = pd.DataFrame()
#solo stats
df['match-id'] = ''
df['game-date'] = ''
#challenges
df['assist-me-pings'] = ''
df['enemy-missing-pings'] = ''
df['bounty-level'] = ''
df['ability-uses'] = ''
df['bounty-gold'] = ''
df['buffs-stolen'] = ''
df['control-wards-placed'] = ''
df['damage-per-minute'] = ''
df['damage-taken-on-team-%'] = ''
df['deaths-by-enemy-champ'] = ''
df['early-laning-gold-exp-advantage'] = ''
df['epic-monster-steals'] = ''
df['first-turret-killed'] = ''
df['gold-per-minute'] = ''
df['kda'] = ''
df['kpa'] = ''
df['max-cs-advantage-on-lane-opponent'] = ''
df['max-kill-deficit'] = ''
df['outnumbered-kills'] = ''
df['played-champ-select-position'] = ''
df['takedowns'] = ''

df['turret-plates-taken'] = ''
df['turret-damage-dealt'] = ''
df['vision-score-per-minute'] = ''
df['vision-score'] = ''
df['champ-name'] = ''
df['champ-level'] = ''
df['deaths'] = ''
df['first-blood'] = ''
df['first-blood-assist'] = ''
df['gold-earned'] = ''
df['gold-spent'] = ''
df['individual-position'] = ''
df['inhibitors-lost'] = ''
df['kills'] = ''
df['longest-time-alive'] = ''
df['riot-id-game-name'] = ''
df['team-position'] = ''
df['total-damage-dealt'] = ''
df['total-time-spent-dead'] = ''
df['team-id'] = ''
df['win'] = ''
df['game-duration'] = ''
df['game-mode'] = 'Ranked'

In [33]:
def fill_df_with_match_info(df, match, match_id):
    try:
        participants = match['info']['participants']
        for player in participants:
            row = {'game-duration': match['info']['gameDuration'],
                'game-date': str(match['info']['gameCreation']),
                'match-id': match_id,
                'total-time-spent-dead': player['totalTimeSpentDead'],
                'win': player['win'],
                'team-id': player['teamId'],
                'total-damage-dealt': player['totalDamageDealtToChampions'],
                'riot-id-game-name': player['riotIdGameName'],
                'game-mode': match['info']['gameMode'],
                'team-position': player['teamPosition'],
                'longest-time-alive': player['longestTimeSpentLiving'],
                'kills': player['kills'],
                'inhibitors-lost': player['inhibitorsLost'],
                'individual-position': player['individualPosition'],
                'gold-spent': player['goldSpent'],
                'gold-earned': player['goldEarned'],
                'first-blood-assist': player['firstBloodAssist'],
                'first-blood': player['firstBloodKill'],
                'deaths': player['deaths'],
                'champ-level': player['champLevel'],
                'champ-name': player['championName'],
                'vision-score': player['visionScore'],
                'vision-score-per-minute': player['challenges']['visionScorePerMinute'],
                'turret-plates-taken': player['challenges']['turretPlatesTaken'],
                'turret-damage-dealt': player['damageDealtToTurrets'],
                'takedowns': player['challenges']['takedowns'],
#                 'played-champ-select-position': player['challenges']['playedChampSelectPosition'],
                'outnumbered-kills': player['challenges']['outnumberedKills'],
                'max-kill-deficit': player['challenges']['maxKillDeficit'],
                'max-cs-advantage-on-lane-opponent': player['challenges']['maxCsAdvantageOnLaneOpponent'],
                'kpa': player['challenges']['killParticipation'],
                'kda': player['challenges']['kda'],
                'gold-per-minute': player['challenges']['goldPerMinute'],
                'first-turret-killed': player['challenges']['firstTurretKilled'],
                'epic-monster-steals': player['challenges']['epicMonsterSteals'],
                'early-laning-gold-exp-advantage': player['challenges']['earlyLaningPhaseGoldExpAdvantage'],
                'deaths-by-enemy-champ': player['challenges']['deathsByEnemyChamps'],
                'damage-taken-on-team-%': player['challenges']['damageTakenOnTeamPercentage'],
                'damage-per-minute': player['challenges']['damagePerMinute'],
                'control-wards-placed': player['challenges']['controlWardsPlaced'],
                'buffs-stolen': player['challenges']['buffsStolen'],
                'bounty-gold': player['challenges']['bountyGold'],
                'ability-uses': player['challenges']['abilityUses'],
                'bounty-level': player['bountyLevel'],
                'assist-me-pings': player['assistMePings'],
                'enemy-missing-pings': str(player['enemyMissingPings'])
            }
            df_dictionary = pd.DataFrame([row])
    #         print(df_dictionary)
            df = pd.concat([df, df_dictionary], ignore_index=True)
    except Exception as e:
        print(e)
    return df


In [34]:
#this gets account puuid: 
player = 'Skeletrex'
account_v1 = f'https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{player}'
res = json.loads(get_req(account_v1, params = None))
puuid = res['puuid']


In [8]:
#get matches from DATE on
now_epoch = int(time.time())
query_epoch = 1689566400 #july, 17th 2023
week_epoch = 604800
count = 100
match_list = []
while(now_epoch > query_epoch):
    res = get_matches(100, puuid, query_epoch)
    print(count, end = ' | ')
    count += 100
    query_epoch += week_epoch
    match_list.extend(res)

100 | 200 | 300 | 400 | 500 | 600 | 700 | 800 | 900 | 1000 | 1100 | 1200 | 1300 | 1400 | 1500 | 1600 | 1700 | 1800 | 1900 | 2000 | 2100 | 2200 | 2300 | 2400 | 2500 | 2600 | 2700 | 2800 | 2900 | 

In [ ]:
#match analysis
#get players matches
count = 1
for match in match_list:
    data = get_match_analytics(match)
    df = fill_df_with_match_info(df, data, match)
    print(count)
    count+=1

'info'
1
'info'
2
'info'
3
'info'
4
5
6
7
8
9
10
11
12
'maxCsAdvantageOnLaneOpponent'
13
'maxCsAdvantageOnLaneOpponent'
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
'riotIdGameName'
40
'riotIdGameName'
41
'riotIdGameName'
42
'riotIdGameName'
43
'riotIdGameName'
44
'riotIdGameName'
45
'riotIdGameName'
46
'riotIdGameName'
47
'riotIdGameName'
48
'riotIdGameName'
49
'riotIdGameName'
50
'riotIdGameName'
51
'riotIdGameName'
52
'riotIdGameName'
53
'riotIdGameName'
54
'riotIdGameName'
55
'riotIdGameName'
56
'riotIdGameName'
57
'riotIdGameName'
58
'riotIdGameName'
59
'riotIdGameName'
60
'riotIdGameName'
61
'riotIdGameName'
62
'riotIdGameName'
63
'riotIdGameName'
64
'riotIdGameName'
65
'riotIdGameName'
66
'riotIdGameName'
67
'riotIdGameName'
68
'riotIdGameName'
69
'riotIdGameName'
70
'riotIdGameName'
71
'riotIdGameName'
72
'riotIdGameName'
73
'riotIdGameName'
74
'riotIdGameName'
75
'riotIdGameName'
76
'riotIdGameName'
77
'riotIdGameName'
78
'riotIdGameName'
79
'r

'riotIdGameName'
655
'riotIdGameName'
656
'riotIdGameName'
657
'riotIdGameName'
658
'riotIdGameName'
659
'riotIdGameName'
660
'riotIdGameName'
661
'info'
662
'info'
663
'info'
664
'info'
665
'info'
666
'info'
667
'info'
668
'info'
669
'info'
670
'info'
671
'info'
672
'info'
673
'info'
674
'info'
675
'info'
676
'info'
677
'info'
678
'info'
679
'info'
680
'info'
681
'info'
682
'info'
683
'info'
684
'info'
685
'info'
686
'info'
687
'info'
688
'info'
689
'info'
690
'info'
691
'info'
692
'info'
693
'info'
694
'info'
695
'info'
696
'info'
697
'info'
698
'info'
699
'info'
700
'info'
701
'info'
702
'info'
703
'info'
704
'info'
705
'info'
706
'info'
707
'info'
708
'info'
709
'info'
710
'info'
711
'info'
712
'info'
713
'info'
714
'info'
715
'info'
716
'info'
717
'info'
718
'info'
719
'info'
720
'info'
721
'info'
722
'info'
723
'info'
724
'info'
725
'info'
726
'info'
727
'info'
728
'info'
729
'info'
730
'info'
731
'info'
732
'info'
733
'info'
734
'info'
735
'info'
736
'info'
737
'info'
738
'info'

In [27]:
df.to_csv("Colins-match-data.csv", index=False))

In [36]:
df.head(10)

,match-id,game-date,assist-me-pings,enemy-missing-pings,bounty-level,ability-uses,bounty-gold,buffs-stolen,control-wards-placed,damage-per-minute,...,kills,longest-time-alive,riot-id-game-name,team-position,total-damage-dealt,total-time-spent-dead,team-id,win,game-duration,game-mode
0,NA1_4910482414,1706911478901,1,0,0,406,0,0,2,766.194844,...,8,668,Try2Hide,TOP,30410,311,100,True,2381,CLASSIC
1,NA1_4910482414,1706911478901,2,3,0,414,0,2,4,362.356764,...,2,955,SkyGenMagician,JUNGLE,14382,92,100,True,2381,CLASSIC
2,NA1_4910482414,1706911478901,3,5,3,256,500,0,1,991.7601,...,6,560,MuddaMin,MIDDLE,39363,125,100,True,2381,CLASSIC
3,NA1_4910482414,1706911478901,2,0,2,200,450,0,0,1001.678832,...,17,1194,MorgothBauglier,BOTTOM,39757,184,100,True,2381,CLASSIC
4,NA1_4910482414,1706911478901,2,17,0,165,0,0,4,366.466843,...,0,916,Captain Inthano,UTILITY,14545,134,100,True,2381,CLASSIC
5,NA1_4910482414,1706911478901,6,14,0,321,0,0,4,703.135417,...,2,460,ßºx ßºx,TOP,27908,454,200,False,2381,CLASSIC
6,NA1_4910482414,1706911478901,13,0,2,520,450,1,3,741.16649,...,4,788,SKELETREX,JUNGLE,29417,101,200,False,2381,CLASSIC
7,NA1_4910482414,1706911478901,3,3,0,153,1450,0,0,1173.027155,...,14,1138,spacemattress,MIDDLE,46558,211,200,False,2381,CLASSIC
8,NA1_4910482414,1706911478901,6,10,0,350,0,0,4,307.139623,...,1,382,GeneralChowMane,BOTTOM,12190,358,200,False,2381,CLASSIC
9,NA1_4910482414,1706911478901,4,9,0,263,0,0,2,564.823715,...,2,893,Cpt Bushwhacker,UTILITY,22418,87,200,False,2381,CLASSIC
